# Introduction
This competition is about identifying steel defects by the means of *semantic segmentation*. 

In this notebook we are going to perform an Exploratory Data Analysis (EDA).

In [ ]:
import numpy as np
import pandas as pd


from os.path import join

from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

import cv2

from tqdm import tqdm

import plotly
from plotly import graph_objects as go
import plotly.express as px


# Main Configuration

In [ ]:
DATA_DIR = join('..', 'input')

TRAIN_IMG_DIR = join(DATA_DIR, 'train_images')
TEST_IMG_DIR = join(DATA_DIR, 'test_images')
SAMPLE_SUB = join(DATA_DIR, 'sample_submission.csv')
TRAIN_DATA = join(DATA_DIR, 'train.csv')
model_save_path = join('.', 'ResUNetSteel_z.h5')
pretrained_model_path = join('..', 'input', 'severstal-pretrained-model', 'ResUNetSteel_z.h5')


train_df = pd.read_csv(TRAIN_DATA)
train_df['ImageId_ClassId'] = train_df.apply(lambda x: '{}_{}'.format(x.ImageId, x.ClassId), axis=1)
sub_df = pd.read_csv(SAMPLE_SUB)
save_model = True


# Kernel Configurations
make_submission = False # used to turn off lengthy model analysis so a submission version doesn't run into memory error
load_pretrained_model = True # load a pre-trained model

# Exploratory Data Analysis
The training data is presented in the following format:

In [ ]:
train_df.head()

In [ ]:
class_counts = {}
class_to_images = {}
for name, group in train_df.groupby(by='ClassId'):
    class_to_images[name] = group['ImageId'].tolist()
    class_counts[name] = len(group)

fig, ax = plt.subplots()
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()), ax=ax)
ax.set_title("Number of images for each class")
ax.set_xlabel("class")
class_counts

You can clearly notice a class imballance present in the dataset. Lets check how many of the images does not have labels at all.

In [ ]:
import os

unique_labeled_imgs = set(train_df.ImageId.unique())
unique_imgs = set(os.listdir(TRAIN_IMG_DIR))

class_to_images['no_damage'] = list(unique_imgs.difference(unique_labeled_imgs))

print('Number of all images:', len(unique_imgs))
print('Number of labeled images:', len(unique_labeled_imgs))
print('Number of unlabeled images:', len(unique_imgs) - len(unique_labeled_imgs))
print('Number of entries in the dataset that does not have an image', len(unique_labeled_imgs.difference(unique_imgs)))
print('Number of images that does not have an entry', len(unique_imgs.difference(unique_labeled_imgs)))
print('Number of test images:', len(sub_df))

There are a lot of images without any defect masks. We are not sure if these images just missing labels or these are examples of instances without any defect. We visualize couple of images per each class to get a better understanding what kind of data is there. We are going to use different collors for each of the deffect classes.

In [ ]:
def load_img(img_id):
    img = cv2.imread(join(TRAIN_IMG_DIR, img_id))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

In [ ]:
rgb_for_label = {i:v for i, v in enumerate([(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)], start=1)}

    
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
for i in range(0, 4):
    ax[i].axis('off')
    ax[i].imshow(np.ones((50, 50, 3), dtype=np.uint8) * rgb_for_label[i+1])
    ax[i].set_title("class color: {}".format(i+1))
fig.suptitle("Colors for the classes")

plt.show()


## Run-Length Encoding
> In order to reduce the submission file size, our metric uses run-length encoding on the pixel values. Instead of submitting an exhaustive list of indices for your segmentation, you will submit **pairs of values** that contain a **start position and a run length**. E.g. '1 3' implies starting at pixel 1 and running a total of 3 pixels (1,2,3).
>
>The competition format requires a **space delimited list of pairs**. For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask. The metric checks that the pairs are **sorted, positive, and the decoded pixel values are not duplicated**. The pixels are numbered from top to bottom, then left to right: 1 is pixel (1,1), 2 is pixel (2,1), etc.

So, if we were to encode something like our example above, we would have to write it as follows:

In [ ]:
# a more elaborate version of kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
# note that we will transpose the incoming array outside of the function, 
# as I find this a clearer illustration

def mask_to_rle(mask):
    """
    params:  mask - numpy array
    returns: run-length encoding string (pairs of start & length of encoding)
    """
    
    # turn a n-dimensional array into a 1-dimensional series of pixels
    # for example:
    #     [[1. 1. 0.]
    #      [0. 0. 0.]   --> [1. 1. 0. 0. 0. 0. 1. 0. 0.]
    #      [1. 0. 0.]]
    flat = mask.flatten()
    
    # we find consecutive sequences by overlaying the mask
    # on a version of itself that is displaced by 1 pixel
    # for that, we add some padding before slicing
    padded = np.concatenate([[0], flat, [0]])
    
    # this returns the indices where the sliced arrays differ
    runs = np.where(padded[1:] != padded[:-1])[0] 
    # indexes start at 0, pixel numbers start at 1
    runs += 1

    # every uneven element represents the start of a new sequence
    # every even element is where the run comes to a stop
    # subtract the former from the latter to get the length of the run
    runs[1::2] -= runs[0::2]
 
    # convert the array to a string
    return ' '.join(str(x) for x in runs)

In [ ]:
def rle_to_mask(lre, shape=(1600, 256)):
    '''
    params:  rle   - run-length encoding string (pairs of start & length of encoding)
             shape - (width,height) of numpy array to return 
    
    returns: numpy array with dimensions of shape parameter
    '''    
    # the incoming string is space-delimited
    runs = np.asarray([int(run) for run in lre.split(' ')])
    
    # we do the same operation with the even and uneven elements, but this time with addition
    runs[1::2] += runs[0::2]
    # pixel numbers start at 1, indexes start at 0
    runs -= 1
    
    # extract the starting and ending indeces at even and uneven intervals, respectively
    run_starts, run_ends = runs[0::2], runs[1::2]
    
    # build the mask
    h, w = shape
    mask = np.zeros(h*w, dtype=np.uint8)
    for start, end in zip(run_starts, run_ends):
        mask[start:end] = 1
    
    # transform the numpy array from flat to the original image shape
    return mask.reshape(shape).T

In [ ]:
def show_masked_image(img_id, ax=None, thickness=2):
    if ax is None:
        fig, ax = plt.subplots(figsize=(15, 5))
    
    img = load_img(img_id)
    for i, col in train_df[train_df['ImageId'] == img_id].iterrows():
        encoded_pixels = col['EncodedPixels']
        label = col['ClassId']
        mask = rle_to_mask(encoded_pixels, shape=(1600, 256))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        img = cv2.drawContours(img, contours, -1, rgb_for_label[label], thickness=thickness)
    ax.imshow(img)
    return ax

def show_masked_images_per_class(label, num_images=5):

    num_imgs = 5
    fig, axs = plt.subplots(nrows=num_imgs, ncols=1, figsize=(15, 15))
    axs = axs.ravel()
    
    image_ids = np.asarray(class_to_images[label])
    random_ids = list(np.random.randint(len(image_ids), size=num_imgs))
    
    for i, img_id in enumerate(image_ids[random_ids]):
        show_masked_image(img_id, ax=axs[i])
        

### Class 1

In [ ]:
show_masked_images_per_class(1, num_images=5)

### Class 2

In [ ]:
show_masked_images_per_class(2, num_images=5)

### Class 3

In [ ]:
show_masked_images_per_class(3, num_images=5)

### Class 4

In [ ]:
show_masked_images_per_class(4, num_images=5)

### Images with no defect

In [ ]:
show_masked_images_per_class('no_damage', num_images=5)

You might notice that these defects differ a lot in the size and number.

### Mask Size Per Defect Class
Since we have binary mask, we will count the number of pixels we have in our mask to get some sort of approximation for the size of defect per class, and look how this varies from class to class.

In [ ]:
# calculate sum of the pixels for the mask per class id
train_df['ClassId_str'] = train_df['ClassId'].astype(str)

train_df['mask_pixel_sum'] = train_df.apply(lambda x: rle_to_mask(x['EncodedPixels']).sum(), axis=1)

class_ids = [str(i) for i in range(1, 5)]
mask_count_per_class = [train_df[(train_df['ClassId_str']==class_id)&(train_df['mask_pixel_sum']!=0)]['mask_pixel_sum'].count() for class_id in class_ids]
pixel_sum_per_class = [train_df[(train_df['ClassId_str']==class_id)&(train_df['mask_pixel_sum']!=0)]['mask_pixel_sum'].sum() for class_id in class_ids]

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = plotly.subplots.make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=class_ids, values=mask_count_per_class, name="Mask Count"), 1, 1)
fig.add_trace(go.Pie(labels=class_ids, values=pixel_sum_per_class, name="Pixel Count"), 1, 2)
# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Steel Defect Mask & Pixel Count",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Count', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Sum', x=0.80, y=0.5, font_size=20, showarrow=False)])
fig['layout'].update(height=400, width=900, title='Pixel count and sum per class mask', legend={'traceorder':'normal'})
fig.show()

In [ ]:
# plot a histogram and boxplot combined of the mask pixel sum per class Id
fig = px.histogram(train_df[train_df['mask_pixel_sum']!=0][['ClassId','mask_pixel_sum']], 
                   x="mask_pixel_sum", y="ClassId", color="ClassId", marginal="box")

fig['layout'].update(height=400, width=900,title='Histogram and Boxplot of Sum of Mask Pixels Per Class')
fig.show()

From the box plot we can reconfirm our previous observation of class 4 are generally larger in size than class 3, and of course class 1 and 2. Defect class 3 has a lot of outliers. Even though class 4 is generally bigger in size, the outlier values in class 3 can be a lot larger than the ones in class 4!



### Segments Per Defect Type
When we visualize the defects, we can see that per defect we can have multiple regions in our image with the same kind of defect. In this section we find out the number of segments behave for different class of defects.

In [ ]:
def count_segments(mask):
    """Given a mask, count the number of regions.

    Parameters:
    mask (numpy.array): numpy array of the mask

    Returns:
    int: number of segments
    """
    # if the mask is empty return zero
    if mask.sum() == 0:
        return 0
    else:
        # use open cv and threshold mechanism to calculate contours
        _, contours = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # get the segment count
        for c in contours:
            segments_count = len(c)

        return segments_count

In [ ]:
# use the count_segments function to conver encoded mask to mask and count the number of segments per defect
train_df['segments'] = train_df.apply(lambda r: count_segments(rle_to_mask(r['EncodedPixels'])),axis=1)

# use the count_segments function to conver encoded mask to mask and count the number of segments per defect
train_df['avg_mask_per_seg'] = (train_df['mask_pixel_sum'] / train_df['segments']).fillna(0)

In [ ]:

fig = px.scatter(train_df[train_df['mask_pixel_sum']!=0], x="mask_pixel_sum", y="segments", color='ClassId_str', size="avg_mask_per_seg", hover_data=["avg_mask_per_seg"])
fig['layout'].update(height=800, width=800,title='')
fig.show()

In [ ]:
fig = px.scatter(train_df[train_df['mask_pixel_sum']!=0], x="mask_pixel_sum", y="segments", color="ClassId_str", marginal_y="rug", marginal_x="histogram")
fig['layout'].update(height=800, width=800,title='')
fig.show()

<a id="1"></a> <br>

## Prediction Output Format
The following explanation is taken from here: https://www.kaggle.com/robinteuwens/mask-rcnn-detailed-starter-code

From the competition's [data](https://www.kaggle.com/c/severstal-steel-defect-detection/data) page:
> Each image may have no defects, a defect of a single class, or defects of multiple classes. For each image you must segment defects of each class ```(ClassId = [1, 2, 3, 4])```.

The submission format requires us to make the classifications for each respective class on a separate row, adding the *_class* to the imageId:
![format](https://i.imgur.com/uEeoOQg.png)

## Loss Function

### Dice Coefficient
From the [evaluation](https://www.kaggle.com/c/severstal-steel-defect-detection/overview/evaluation) page:

> This competition is evaluated on the mean Dice coefficient. The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth. The formula is given by:
>
>$$Dice(X,Y) = \frac{2∗|X∩Y|}{|X|+|Y|}$$
>
>
>where X is the predicted set of pixels and Y is the ground truth. The Dice coefficient is defined to be 1 when both X and Y are empty. The leaderboard score is the mean of the Dice coefficients for each ```<ImageId, ClassId>``` pair in the test set.

Visual illustration of the Dice Coefficient:
![dice_viz](https://i.imgur.com/zl2W0xQ.png)



In [ ]:
sample_size = 100
image_shapes = train_df.iloc[np.random.randint(len(train_df), size=sample_size), :].ImageId.apply(lambda x: load_img(x).shape)
print(len(image_shapes.unique()))
print(image_shapes.iloc[0])

image_shape = IMAGE_X, IMAGE_Y, IMAGE_CHANNELS = image_shapes.iloc[0]

# Summary
Taking into the account the class imballances and the size of the mask patches it seems like the regular crossentropy loss will not do a good job on this dataset.

![image.png](https://i.ibb.co/bHPQrmR/image-2021-01-15-150827.png)
Source: https://arxiv.org/pdf/2006.14822.pdf
